In [1]:
import pandas as pd

In [2]:
excel_file_path = 'C:/Users/ASUS/Documents/實習/Rebas/CPBL_2022.xlsx'  # 請替換成你的 Excel 檔案路徑
sheet_name1 = 'AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 = 'HOME-打席資訊'

In [3]:
away = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
home = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

In [4]:
cols = ['客場','主場','打席結果','落點1','落點2','彈道']

In [5]:
away_play = away[cols]
home_play = home[cols]

## 客隊投手

In [6]:
def calculate_so_go_count(series):
    so_count = (series == 'SO').sum()
    ubb_count = (series == 'uBB').sum()
    hbp_count = (series == 'HBP').sum()
    hr_count = (series == 'HR').sum()
    return pd.Series({'SO': so_count,
                      'HR':hr_count
                     , 'uBB':ubb_count
                     , 'HBP':hbp_count})

In [7]:
resault_home=home_play .groupby(['客場'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_home = resault_home.rename(columns={'level_1': '打席結果', '打席結果': '次數'})
resault_home

,客場,打席結果,次數
0,中信兄弟,SO,382
1,中信兄弟,HR,16
2,中信兄弟,uBB,138
3,中信兄弟,HBP,42
4,味全龍,SO,367
5,味全龍,HR,18
6,味全龍,uBB,189
7,味全龍,HBP,35
8,富邦悍將,SO,366
9,富邦悍將,HR,19


In [8]:
resault_home['次數'] = resault_home['次數'].astype(int)

final_resault_home = pd.pivot_table(resault_home, index=['客場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_resault_home

打席結果,HBP,HR,SO,uBB
客場,,,,
中信兄弟,42,16,382,138
味全龍,35,18,367,189
富邦悍將,42,19,366,150
樂天桃猿,24,25,409,161
統一7-ELEVEn獅,22,27,376,155


In [9]:
def classify_fly_ball(row):
    if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        return '外野飛球'
    elif row['彈道'] == 'F' and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '內野飛球'
    elif row['彈道'] == 'P' and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

home_play['飛球分類'] = home_play.apply(classify_fly_ball, axis=1)
away_play['飛球分類'] = away_play.apply(classify_fly_ball, axis=1)
# 打印结果


C:\Users\ASUS\AppData\Local\Temp\ipykernel_20304\3959518264.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_play['飛球分類'] = home_play.apply(classify_fly_ball, axis=1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20304\3959518264.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  away_play['飛球分類'] = away_play.apply(classify_fly_ball, axis=1)


In [10]:
def calculate_so_go_count2(series):
    iff_count = (series == '內野飛球').sum()
    off_count = (series == '外野飛球').sum()
    ld_count = (series == '平飛球').sum()
    gr_count = (series == '滾地球').sum()
    return pd.Series({'內野飛球': iff_count,
                      '外野飛球':off_count
                     , '平飛球':ld_count
                     , '滾地球':gr_count})

In [11]:
resault_home2=home_play.groupby(['客場'])['飛球分類'].apply(calculate_so_go_count2).reset_index()
resault_home2 = resault_home2.rename(columns={'level_1': '擊球分類', '飛球分類': '次數'})

resault_home2['次數'] = resault_home2['次數'].astype(int)

final_resault_home2 = pd.pivot_table(resault_home2, index=['客場'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault_home2

擊球分類,內野飛球,外野飛球,平飛球,滾地球
客場,,,,
中信兄弟,600,134,236,736
味全龍,543,136,224,769
富邦悍將,549,138,213,754
樂天桃猿,512,102,214,769
統一7-ELEVEn獅,567,118,225,799


In [12]:
pitcher_home = pd.merge(final_resault_home, final_resault_home2, on='客場', how='inner')

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
客場,,,,,,,,
中信兄弟,42,16,382,138,600,134,236,736
味全龍,35,18,367,189,543,136,224,769
富邦悍將,42,19,366,150,549,138,213,754
樂天桃猿,24,25,409,161,512,102,214,769
統一7-ELEVEn獅,22,27,376,155,567,118,225,799


In [21]:
constant=2.8693797931401934
pitcher_home['tRA']=((pitcher_home ['uBB']*0.311522
                 +pitcher_home ['HBP']*0.328145
                 -pitcher_home ['SO']*0.27208
                 +pitcher_home ['HR']*1.657671
                 -pitcher_home ['滾地球']*0.0554
                 +pitcher_home ['內野飛球']*0.065453
                 -pitcher_home ['外野飛球']*0.26825
                 +pitcher_home ['平飛球']*0.327159)/
                (pitcher_home ['SO']
                 +pitcher_home ['滾地球']*0.712091
                +pitcher_home ['平飛球']*0.261841
                +pitcher_home ['內野飛球']*0.710578
                +pitcher_home ['外野飛球']*0.982011)*27)+constant

In [22]:
pitcher_home

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
客場,,,,,,,,,
中信兄弟,42,16,382,138,600,134,236,736,3.207744
味全龍,35,18,367,189,543,136,224,769,3.413621
富邦悍將,42,19,366,150,549,138,213,754,3.219602
樂天桃猿,24,25,409,161,512,102,214,769,3.267107
統一7-ELEVEn獅,22,27,376,155,567,118,225,799,3.450371


## 主隊投手

In [13]:
#groupby(['主場']) 切換算主客場
resault_away=away_play .groupby(['主場'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_away = resault_away.rename(columns={'level_1': '打席結果', '打席結果': '次數'})

resault_away['次數'] = resault_away['次數'].astype(int)

final_resault_away = pd.pivot_table(resault_away, index=['主場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_resault_away

打席結果,HBP,HR,SO,uBB
主場,,,,
中信兄弟,28,29,437,130
味全龍,33,25,353,177
富邦悍將,30,30,430,169
樂天桃猿,35,27,390,141
統一7-ELEVEn獅,20,36,404,174


In [14]:
##groupby(['主場']) 切換算主客場
resault_away2=away_play.groupby(['主場'])['飛球分類'].apply(calculate_so_go_count2).reset_index()
resault_away2 = resault_away2.rename(columns={'level_1': '擊球分類', '飛球分類': '次數'})

resault_away2['次數'] = resault_away2['次數'].astype(int)

final_resault_away2 = pd.pivot_table(resault_away2, index=['主場'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault_away2

擊球分類,內野飛球,外野飛球,平飛球,滾地球
主場,,,,
中信兄弟,550,104,205,786
味全龍,519,126,253,830
富邦悍將,531,102,260,853
樂天桃猿,551,117,215,899
統一7-ELEVEn獅,627,146,193,745


In [15]:
pitcher_away = pd.merge(final_resault_away, final_resault_away2, on='主場', how='inner')

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
主場,,,,,,,,
中信兄弟,28,29,437,130,550,104,205,786
味全龍,33,25,353,177,519,126,253,830
富邦悍將,30,30,430,169,531,102,260,853
樂天桃猿,35,27,390,141,551,117,215,899
統一7-ELEVEn獅,20,36,404,174,627,146,193,745


In [19]:
constant=2.8693797931401934
pitcher_away ['tRA']=((pitcher_away ['uBB']*0.311522
                 +pitcher_away ['HBP']*0.328145
                 -pitcher_away ['SO']*0.27208
                 +pitcher_away ['HR']*1.657671
                 -pitcher_away ['滾地球']*0.0554
                 +pitcher_away ['內野飛球']*0.065453
                 -pitcher_away ['外野飛球']*0.26825
                 +pitcher_away ['平飛球']*0.327159)/
                (pitcher_away ['SO']
                 +pitcher_away ['滾地球']*0.712091
                +pitcher_away ['平飛球']*0.261841
                +pitcher_away ['內野飛球']*0.710578
                +pitcher_away ['外野飛球']*0.982011)*27)+constant

In [20]:
pitcher_away 

,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
主場,,,,,,,,,
中信兄弟,28,29,437,130,550,104,205,786,3.052790
味全龍,33,25,353,177,519,126,253,830,3.737616
富邦悍將,30,30,430,169,531,102,260,853,3.558914
樂天桃猿,35,27,390,141,551,117,215,899,3.196005
統一7-ELEVEn獅,20,36,404,174,627,146,193,745,3.457203


In [16]:
#pitcher_away.to_excel('客場球隊投手各項目總數.xlsx', index=True)

In [17]:
def calculate_so_go_count3(series):
    so_count = (series == 'SO').sum()
    go_count = (series == 'GO').sum()
    gidp_count = (series == 'GIDP').sum()
    fo_count = (series == 'FO').sum()
    sh_count = (series == 'SH').sum()
    sf_count = (series == 'SF').sum()
    hit_count = (series == '1B').sum()
    hit2_count = (series == '2B').sum()
    hit3_count = (series == '3B').sum()
    hit4_count = (series == 'HR').sum()
    ubb_count = (series == 'uBB').sum()
    e_count = (series == 'E').sum()
    fc_count = (series == 'FC').sum()
    hbp_count = (series == 'HBP').sum()
    she_count = (series == 'SH_E').sum()
    shfc_count = (series == 'SH_FC').sum()
    return pd.Series({'SO': so_count, 'GO': go_count ,'GIDP':gidp_count, 'FO':fo_count, 'SH':sh_count, 'SF':sf_count, '1B':hit_count
                     , '2B':hit2_count
                     , '3B':hit3_count
                     , 'HR':hit4_count
                     , 'uBB':ubb_count
                     , 'E':e_count
                     , 'FC':fc_count
                     , 'HBP':hbp_count
                     , 'SH_E':she_count
                     , 'SH_FC':shfc_count})

In [18]:
#groupby(['主場']) 切換算主客場
away_count=away_play.groupby(['主場'])['打席結果'].apply(calculate_so_go_count3).reset_index()
away_count =away_count.rename(columns={'level_1': '打席結果', '打席結果': '次數'})

away_count ['次數'] =away_count ['次數'].astype(int)

final_count_away = pd.pivot_table(away_count , index=['主場'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_count_away

打席結果,1B,2B,3B,E,FC,FO,GIDP,GO,HBP,HR,SF,SH,SH_E,SH_FC,SO,uBB
主場,,,,,,,,,,,,,,,,
中信兄弟,394,72,4,21,77,521,47,437,28,29,20,24,0,0,437,130
味全龍,376,88,10,32,77,559,39,459,33,25,18,41,4,1,353,177
富邦悍將,429,84,10,23,85,521,49,476,30,30,16,21,2,1,430,169
樂天桃猿,472,74,5,24,87,540,53,446,35,27,20,35,2,1,390,141
統一7-ELEVEn獅,396,88,7,34,56,598,49,390,20,36,22,32,3,0,404,174
